In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
batch_size = 32
img_height = 256
img_width = 256

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  r"Dataset v5",
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  label_mode='int',
  batch_size=batch_size)

Found 3500 files belonging to 7 classes.
Using 2800 files for training.


In [4]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  r"Dataset v5",
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  label_mode='int',
  batch_size=batch_size)

Found 3500 files belonging to 7 classes.
Using 700 files for validation.


In [5]:
class_names = train_ds.class_names
print(class_names)

['10', '100', '20', '200', '2000', '50', '500']


In [6]:
# Mapping class names to integer labels
class_to_label = {class_name: label for label, class_name in enumerate(class_names)}

# Dummy labels 
labels = ['10', '100', '20', '200', '2000', '50', '500']
integer_labels = [class_to_label[label] for label in labels]

In [7]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
preprocess_input = tf.keras.applications.resnet.preprocess_input

In [9]:
img_size = (256, 256)
img_shape = img_size + (3,)
base_model = tf.keras.applications.ResNet50(input_shape=img_shape,
                                               include_top=False,
                                               weights='imagenet')

In [10]:
base_model.trainable = False

In [11]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [12]:
image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 8, 8, 2048)


In [13]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 2048)


In [14]:
prediction_layer = tf.keras.layers.Dense(7, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 7)


In [15]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),  # Randomly flip horizontally
    tf.keras.layers.RandomRotation(20),        # Randomly rotate images (up to 20 degrees)
    tf.keras.layers.RandomZoom(0.1),           # Randomly zoom images (up to 10%)
    tf.keras.layers.RandomBrightness(0.1),     # Randomly adjust brightness (up to 10%)
    tf.keras.layers.RandomContrast(0.1),       # Randomly adjust contrast (up to 10%)
])

In [16]:
inputs = tf.keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 tf.__operators__.getitem (S  (None, 256, 256, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 256, 256, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                      

In [18]:
import os
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [19]:
initial_learning_rate = 1e-5
# Create a learning rate schedule
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=1000,  # Decay every 1000 steps/epochs
    decay_rate=0.9,     # Decay rate
    staircase=True)     # Decay in discrete steps

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])


In [21]:
from tensorflow.keras.callbacks import EarlyStopping
history = model.fit(train_ds,
                    epochs=150,
                    callbacks=[EarlyStopping(patience=3, restore_best_weights=True),cp_callback],
                    validation_data=val_ds)

Epoch 1/150
88/88 [==============================] - ETA: 0s - loss: 2.3468 - sparse_categorical_accuracy: 0.1764
Epoch 1: saving model to training_1\cp.ckpt
88/88 [==============================] - 360s 4s/step - loss: 2.3468 - sparse_categorical_accuracy: 0.1764 - val_loss: 2.2055 - val_sparse_categorical_accuracy: 0.1629
Epoch 2/150
88/88 [==============================] - ETA: 0s - loss: 2.2106 - sparse_categorical_accuracy: 0.1964
Epoch 2: saving model to training_1\cp.ckpt
88/88 [==============================] - 319s 4s/step - loss: 2.2106 - sparse_categorical_accuracy: 0.1964 - val_loss: 2.0723 - val_sparse_categorical_accuracy: 0.1857
Epoch 3/150
88/88 [==============================] - ETA: 0s - loss: 2.0554 - sparse_categorical_accuracy: 0.2346
Epoch 3: saving model to training_1\cp.ckpt
88/88 [==============================] - 311s 4s/step - loss: 2.0554 - sparse_categorical_accuracy: 0.2346 - val_loss: 1.9816 - val_sparse_categorical_accuracy: 0.2043
Epoch 4/150
88/88 [===

In [22]:
model.save("ResNet.keras")

In [23]:
model.save("ResNet.h5")

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('NewModel.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\nehul\AppData\Local\Temp\tmpt9asicc1\assets


INFO:tensorflow:Assets written to: C:\Users\nehul\AppData\Local\Temp\tmpt9asicc1\assets


In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.resnet import preprocess_input
from PIL import Image, ImageDraw, ImageFont

# Load your trained model
model = load_model('ResNet.keras')  # Adjust the path to where your model is stored

def load_and_preprocess_image(image_path):
    """ Load and preprocess an image from the file path """
    img = Image.open(image_path).convert('RGB')
    img = img.resize((256, 256))  # Resize image to match model's expected input shape
    img_array = img_to_array(img)  # Convert the image to a numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
    img_array = preprocess_input(img_array)  # Preprocess the image
    return img, img_array

def predict_image(model, img_array):
    """ Predict the class of an image """
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions)
    return class_names[predicted_class], confidence

def display_image_with_prediction(img, predicted_class, confidence):
    """ Display image with predicted class and confidence """
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()  # Load default font
    text = f'{predicted_class} ({confidence*100:.2f}%)'
    draw.text((10, 10), text, font=font, fill='red')  # Draw text on top left corner
    img.show()

# Example usage:
if __name__ == "__main__":
    image_path = r"C:\Users\nehul\OneDrive\Desktop\Project\Dataset v2\100 Rupees Note\0_100IndianRupees_5_jpeg.rf.c0622783b850b3766513ff1532a70459.jpg" # Replace with your image path
    img, img_processed = load_and_preprocess_image(image_path)
    predicted_class, confidence = predict_image(model, img_processed)
    display_image_with_prediction(img, predicted_class, confidence)


1/1 [==============================] - 1s 1s/step


In [11]:
import cv2
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model("Model_v2.keras")

cap = cv2.VideoCapture(0)  # '0' is typically the built-in webcam
if not cap.isOpened():
    print("Cannot open camera")
    exit()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    
    # Resize frame to match model's expected input size, e.g., 224x224
    input_frame = cv2.resize(frame, (256, 256))
    
    # Convert the frame to a format suitable for the model
    input_frame = np.expand_dims(input_frame, axis=0)
    input_frame = tf.keras.applications.resnet50.preprocess_input(input_frame)

    # Make prediction
    predictions = model.predict(input_frame)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions) * 100  # Confidence percentage

    # Display the resulting frame with the prediction
    label = f"Class: {predicted_class}, Confidence: {confidence:.2f}%"
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('frame', frame)

    # Break the loop with 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━

In [6]:
import cv2
import numpy as np
import tensorflow as tf

class_names = ['10', '100', '20', '200', '2000', '50', '500']

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='NewModel.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Configure camera settings
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera could not be opened.")
    exit()

# Function to preprocess the frame
def preprocess_frame(frame, size):
    frame_resized = cv2.resize(frame, size)  # Adjust size to match model's expected input
    frame_preprocessed = tf.keras.applications.resnet.preprocess_input(frame_resized)
    return frame_preprocessed

# Prediction loop
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame (stream end?). Exiting ...")
            break

        # Preprocess the image to fit the model input
        processed_frame = preprocess_frame(frame, (input_details[0]['shape'][2], input_details[0]['shape'][1]))
        processed_frame = np.expand_dims(processed_frame, axis=0)  # Add batch dimension

        # Set input tensor
        interpreter.set_tensor(input_details[0]['index'], processed_frame.astype(np.float32))

        # Run inference
        interpreter.invoke()

        # Extract the output and display the prediction
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predictions = tf.nn.softmax(output_data[0])  # Apply softmax to logits
        predicted_class = class_names[np.argmax(predictions)]
        confidence = np.max(predictions) * 100  # Max probability as confidence

        # Display predicted class and confidence on the frame
        if confidence < 19.5:
            label = "No currency detected"
        else:
            label = f"Note: {predicted_class} Rupees   Confidence: {confidence:.2f}%"
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) == ord('q'):
            break
finally:
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import pyttsx3

class_names = ['10', '100', '20', '200', '2000', '50', '500']

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='NewModel.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Configure camera settings
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera could not be opened.")
    exit()

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Function to preprocess the frame
def preprocess_frame(frame, size):
    frame_resized = cv2.resize(frame, size)  # Adjust size to match model's expected input
    frame_preprocessed = tf.keras.applications.resnet.preprocess_input(frame_resized)
    return frame_preprocessed

# Prediction loop
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame (stream end?). Exiting ...")
            break

        # Preprocess the image to fit the model input
        processed_frame = preprocess_frame(frame, (input_details[0]['shape'][2], input_details[0]['shape'][1]))
        processed_frame = np.expand_dims(processed_frame, axis=0)  # Add batch dimension

        # Set input tensor
        interpreter.set_tensor(input_details[0]['index'], processed_frame.astype(np.float32))

        # Run inference
        interpreter.invoke()

        # Extract the output and display the prediction
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predictions = tf.nn.softmax(output_data[0])  # Apply softmax to logits
        predicted_class = class_names[np.argmax(predictions)]
        confidence = np.max(predictions) * 100  # Max probability as confidence

        # Display predicted class and confidence on the frame
        if confidence < 20:
            label = "No currency detected"
        else:
            label = f"Note: {predicted_class} Rupees   Confidence: {confidence:.2f}%"

        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('Frame', frame)

        # Announce the prediction when spacebar is pressed
        if cv2.waitKey(1) == 32:  # ASCII value for spacebar is 32
            if confidence >= 20:
                engine.say(f"{predicted_class} Rupees")
                engine.runAndWait()

except KeyboardInterrupt:
    print("Interrupted by user")

finally:
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    engine.stop()


In [2]:
import cv2
import numpy as np
import tensorflow as tf
import pyttsx3

class_names = ['10', '100', '20', '200', '2000', '50', '500']

# Load TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='NewModel.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Configure camera settings
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Camera could not be opened.")
    exit()

# Initialize text-to-speech engine
engine = pyttsx3.init()

# Function to preprocess the frame
def preprocess_frame(frame, size):
    frame_resized = cv2.resize(frame, size)  # Adjust size to match model's expected input
    frame_preprocessed = tf.keras.applications.resnet.preprocess_input(frame_resized)
    return frame_preprocessed

# Prediction loop
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame (stream end?). Exiting ...")
            break

        # Preprocess the image to fit the model input
        processed_frame = preprocess_frame(frame, (input_details[0]['shape'][2], input_details[0]['shape'][1]))
        processed_frame = np.expand_dims(processed_frame, axis=0)  # Add batch dimension

        # Set input tensor
        interpreter.set_tensor(input_details[0]['index'], processed_frame.astype(np.float32))

        # Run inference
        interpreter.invoke()

        # Extract the output and display the prediction
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predictions = tf.nn.softmax(output_data[0])  # Apply softmax to logits
        predicted_class = class_names[np.argmax(predictions)]
        confidence = np.max(predictions) * 100  # Max probability as confidence

        # Display predicted class and confidence on the frame
        if confidence < 19.5:
            label = "No currency detected"
        else:
            label = f"Note: {predicted_class} Rupees   Confidence: {confidence:.2f}%"

        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('Frame', frame)

        # Announce the prediction when spacebar is pressedq
        if cv2.waitKey(1) == 32:  # ASCII value for spacebar is 32
            if confidence >= 19.5:
                engine.say(f"{predicted_class} Rupees")
            else:
                engine.say("No currency detected")    
            engine.runAndWait()
        if cv2.waitKey(1) == ord('q'):
            break

finally:
    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
    engine.stop()


KeyboardInterrupt: 